In [1]:
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_file.close()

In [3]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [4]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [5]:
def get_name_artists_from_track(track):
    return [(track["name"],artist["name"]) for artist in track["artists"]]

def flatten_to_artist(input_list):
    return [item[1] for sublist in input_list for item in sublist]

def get_name_artist_from_playlist_item(playlist_item):
    return get_name_artists_from_track(playlist_item['track'])

In [6]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        # sleep(randint(100,1000)/1000)
    return tracks

In [7]:
# mytracks = get_playlist_tracks('05Tli1k3x5YQEvNZyOcR4Z') # wixderwoche
mytracks = get_playlist_tracks('4rnleEAOdmFAbRcNCgZMpY')

In [8]:
mytracks_artists = [get_name_artist_from_playlist_item(mytrack) for mytrack in mytracks]

my_artists = flatten_to_artist(mytracks_artists)

# drop duplicate artists
my_artists_unique = list(set(my_artists))
# my_artists_unique = []
# [my_artists_unique.append(artist.strip()) for artist in my_artists if artist.strip() not in my_artists_unique]

display(len(my_artists_unique))


2435

In [9]:
import json

In [10]:
# features = {}   #dict.fromkeys(my_artists_unique)
counter = 0
batch_size = 100

for i in range(0, len(my_artists_unique), batch_size):
    features = {}
    start = i
    stop = i + batch_size

    for my_artist in my_artists_unique[start:stop]:

        print(f'fetching {counter:5.0f}:  ', my_artist)
        counter += 1
        
        try:
            tr = sp.search(q=f'artist:{my_artist}', type='track', limit=20)['tracks']['items']

            if len(tr) > 0:
                my_artist_features = []
                for t in tr:
                    song = {}
                    song['id'] = t['id']
                    song['track_name'] = t['name']
                    song['preview'] = t['preview_url']
                    song['features'] = sp.audio_features(t['uri'])
                    my_artist_features.append(song)
        except:
            print('something went wrong')
            continue

        features[my_artist] = {'number':counter, 'features':my_artist_features}

        # sleep(randint(100,500)/1000)

    with open(f'songs/artists_{start}_{stop}.json', 'w') as json_file:
        json.dump(features, json_file, indent=4)


fetching     0:   
fetching     1:   Fernando
fetching     2:   J Balvin
fetching     3:   Sharon
fetching     4:   Haddaway
fetching     5:   The All-American Rejects
fetching     6:   FROIDZ
fetching     7:   NOTD
fetching     8:   The Outlaws
fetching     9:   Tyla Yaweh
fetching    10:   Lindsay Lohan
fetching    11:   Cobra Starship
fetching    12:   Slash
fetching    13:   Havana Brown
fetching    14:   Olavi Uusivirta
fetching    15:   Morgan Page
fetching    16:   Piso 21
fetching    17:   Jorja Smith
fetching    18:   Yetmir
fetching    19:   Starley
fetching    20:   Janieck
fetching    21:   Krewella
fetching    22:   Geto Boys
fetching    23:   Sami Saari
fetching    24:   Lil Wayne
fetching    25:   Debra Reynolds
fetching    26:   Shortster 0161
fetching    27:   John Martin
fetching    28:   The Bangles
fetching    29:   Ressu Redford
fetching    30:   Ape Drums
fetching    31:   Gala
fetching    32:   Jonas Vogel
fetching    33:   Tommy Trash
fetching    34:   ILOVEMAKO